In [1]:
import cirq
import qsimcirq

# Define qubits and a short circuit.
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.H(q0), cirq.CX(q0, q1))
print("Circuit:")
print(circuit)
print()

# Simulate the circuit with Cirq and return the full state vector.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.simulate(circuit)
print(cirq_results)
print()

# Simulate the circuit with qsim and return the full state vector.
print('qsim results:')
qsim_simulator = qsimcirq.QSimSimulator()
qsim_results = qsim_simulator.simulate(circuit)
print(qsim_results)

Circuit:
0: ───H───@───
          │
1: ───────X───

Cirq results:
measurements: (no measurements)

qubits: (cirq.LineQubit(0), cirq.LineQubit(1))
output vector: 0.707|00⟩ + 0.707|11⟩

phase:
output vector: |⟩

qsim results:
measurements: (no measurements)
output vector: 0.707|00⟩ + 0.707|11⟩


In [2]:
samples = cirq.sample_state_vector(
    qsim_results.state_vector(), indices=[0, 1], repetitions=10)
print(samples)

[[1 1]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]]


In [3]:
# Define a circuit with measurements.
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(q0), cirq.X(q1), cirq.CX(q0, q1),
    cirq.measure(q0, key='qubit_0'),
    cirq.measure(q1, key='qubit_1'),
)
print("Circuit:")
print(circuit)
print()

# Simulate the circuit with Cirq and return just the measurement values.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.run(circuit, repetitions=5)
print(cirq_results)
print()

# Simulate the circuit with qsim and return just the measurement values.
print('qsim results:')
qsim_simulator = qsimcirq.QSimSimulator()
qsim_results = qsim_simulator.run(circuit, repetitions=5)
print(qsim_results)

Circuit:
0: ───H───@───M('qubit_0')───
          │
1: ───X───X───M('qubit_1')───

Cirq results:
qubit_0=00111
qubit_1=11000

qsim results:
qubit_0=01111
qubit_1=10000


In [4]:
import time

# Get a rectangular grid of qubits.
qubits = cirq.GridQubit.rect(4, 5)

# Generates a random circuit on the provided qubits.
circuit = cirq.experiments.random_rotations_between_grid_interaction_layers_circuit(
    qubits=qubits, depth=16)

# Simulate the circuit with Cirq and print the runtime.
cirq_simulator = cirq.Simulator()
cirq_start = time.time()
cirq_results = cirq_simulator.simulate(circuit)
cirq_elapsed = time.time() - cirq_start
print(f'Cirq runtime: {cirq_elapsed} seconds.')
print()

# Simulate the circuit with qsim and print the runtime.
qsim_simulator = qsimcirq.QSimSimulator()
qsim_start = time.time()
qsim_results = qsim_simulator.simulate(circuit)
qsim_elapsed = time.time() - qsim_start
print(f'qsim runtime: {qsim_elapsed} seconds.')

Cirq runtime: 3.095048189163208 seconds.

qsim runtime: 0.08838915824890137 seconds.


In [5]:
# Use eight threads to parallelize simulation.
options = {'t': 8}

qsim_simulator = qsimcirq.QSimSimulator(options)
qsim_start = time.time()
qsim_results = qsim_simulator.simulate(circuit)
qsim_elapsed = time.time() - qsim_start
print(f'qsim runtime: {qsim_elapsed} seconds.')

qsim runtime: 0.039893388748168945 seconds.


In [125]:
def testByNumberOfqubits(noq):
    qs = cirq.LineQubit.range(noq)
    hs = [cirq.H(x) for x in qs]
    ms = [cirq.measure(x) for x in qs]
    circuit = cirq.Circuit(hs + ms)
    qsim_simulator = qsimcirq.QSimSimulator()
    start = time.time()
    for _ in range(128):
        results = qsim_simulator.simulate(circuit)
    elapsed = time.time() - start
    return(elapsed)

for x in range(2,26):
    a = testByNumberOfqubits(x)
    print(f"{x}: {a}")

2: 0.1874988079071045
3: 0.20644831657409668
4: 0.22838926315307617
5: 0.25232458114624023
6: 0.2652902603149414
7: 0.2842395305633545
8: 0.307178258895874
9: 0.32172727584838867
10: 0.33905553817749023
11: 0.3670344352722168
12: 0.38396263122558594
13: 0.4009108543395996
14: 0.42714643478393555
15: 0.4576451778411865
16: 0.5048532485961914
17: 0.6017842292785645
18: 0.7601807117462158
19: 1.0343987941741943
20: 1.9275481700897217
21: 3.6496312618255615
22: 7.031660795211792
23: 13.907992362976074
24: 28.469674348831177
25: 56.71267890930176


In [128]:
def testByDepth(depth):
    qs = cirq.LineQubit.range(4)
    hs = [cirq.H(x) for x in qs]
    lhs = []
    for _ in range(depth):
        lhs = lhs + hs
    ms = [cirq.measure(x) for x in qs]
    circuit = cirq.Circuit(lhs + ms)
    qsim_simulator = qsimcirq.QSimSimulator()
    start = time.time()
    for _ in range(128):
        results = qsim_simulator.simulate(circuit)
    elapsed = time.time() - start
    return(elapsed)

for x in range(1,40,2):
    a = testByDepth(x)
    print(f"{x}: {a}")

1: 0.2355022430419922
3: 0.27062273025512695
5: 0.31475186347961426
7: 0.367006778717041
9: 0.4085361957550049
11: 0.44965219497680664
13: 0.5005624294281006
15: 0.537405252456665
17: 0.5775527954101562
19: 0.6255645751953125
21: 0.6789026260375977
23: 0.7146646976470947
25: 0.760988712310791
27: 0.8308513164520264
29: 0.8796415328979492
31: 0.9001069068908691
33: 0.9682726860046387
35: 0.9976630210876465
37: 1.0380191802978516
39: 1.0825088024139404
